In [1]:

from keras.models import load_model
import cv2
import numpy as np
cv2.__version__

Using TensorFlow backend.
/home/tasneem/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tasneem/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tasneem/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tasneem/.local/lib/python3.6/site-packages/tensorflow/py

'4.1.0'

In [2]:
model = load_model('cnnface_mask_detection_1.h5')

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:

while(True):

    ret,img=source.read()
    #img2 = np.uint8(img)
    #gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(img,1.3,5) 

    for (x,y,w,h) in faces:
    
        face_img=img[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        X = np.array(resized).reshape(-1,100, 100, 1)
        normalized=X/255.0
        #normalized_image = np.uint8(normalized) 
        #reshaped=np.reshape(normalized_image,(1,100,100,1))#np.resize(img, (-1, <image shape>)
        result=model.predict(normalized)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        acc=round(np.max(result,axis=1)[0]*100,2)
        cv2.putText(img, str(acc), (x+150, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2) 
        
        
        
    if ret == True:

        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        cv2.imshow('LIVE',img)
        


        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    else:
        break

source.release()
cv2.destroyAllWindows()    
    